In [110]:
# Import necessary packages
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

from sklearn.neighbors import KNeighborsClassifier 
from sklearn.model_selection import cross_val_score 

In [111]:
path = r'C:\Users\micha\Desktop\Kaggle - Data Science Bowl 2019'
os.chdir(path)

In [112]:
# Load the training data
train = pd.read_csv('train.csv')#, index_col='timestamp', parse_dates=True) #,usecols=keep_cols)
train_labels = pd.read_csv('train_labels.csv')

# Keep only the following columns in the test set to save RAM on the Kaggle Kernel
keep_cols1 = ['installation_id','world','timestamp','type','game_time','game_session',]

# Load the test data
test = pd.read_csv('test.csv', usecols=keep_cols1) # nrows =100000

In [113]:
print('Size of train data', train.shape)
print('Size of train_labels data', train_labels.shape)
print('Size of specs data', specs.shape)
print('Size of test data', test.shape)

Size of train data (11341042, 11)
Size of train_labels data (17690, 7)
Size of specs data (386, 3)
Size of test data (1156414, 6)


In [114]:
## Function to reduce the DF size
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [115]:
%%time
## REducing memory
train = reduce_mem_usage(train)
train_labels = reduce_mem_usage(train_labels)
specs = reduce_mem_usage(specs)
test = reduce_mem_usage(test)

Mem. usage decreased to 778.73 Mb (18.2% reduction)
Mem. usage decreased to  0.49 Mb (48.2% reduction)
Mem. usage decreased to  0.01 Mb (0.0% reduction)
Mem. usage decreased to 48.53 Mb (8.3% reduction)
Wall time: 312 ms


In [116]:
train.info()
train_labels.info()
specs.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11341042 entries, 0 to 11341041
Data columns (total 11 columns):
event_id           object
game_session       object
timestamp          object
event_data         object
installation_id    object
event_count        int16
event_code         int16
game_time          int32
title              object
type               object
world              object
dtypes: int16(2), int32(1), object(8)
memory usage: 778.7+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17690 entries, 0 to 17689
Data columns (total 7 columns):
game_session       17690 non-null object
installation_id    17690 non-null object
title              17690 non-null object
num_correct        17690 non-null int8
num_incorrect      17690 non-null int8
accuracy           17690 non-null float16
accuracy_group     17690 non-null int8
dtypes: float16(1), int8(3), object(3)
memory usage: 501.1+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386 entries, 0 to 385
Data columns (t

In [117]:
# Explore data and see if there are any out of place values, missing values etc.
display(test['type'].value_counts())
display(test['installation_id'].value_counts())
display(test['world'].value_counts())

Game          572260
Activity      466274
Assessment    102627
Clip           15253
Name: type, dtype: int64

7b728c89    21237
5a6f3de8    18279
090fe325    13758
77d5414c    13231
ce08e98b    12416
            ...  
69164a28        1
5d3d2ce3        1
27e272e5        1
d1e3bd8c        1
a87db7ff        1
Name: installation_id, Length: 1000, dtype: int64

MAGMAPEAK       511291
TREETOPCITY     332295
CRYSTALCAVES    311387
NONE              1441
Name: world, dtype: int64

In [118]:
# Check for missing data
assert pd.notna(train).all().all()
assert pd.notna(train_labels).all().all()
assert pd.notna(test).all().all()
assert pd.notna(specs).all().all()
print('No missing data')

No missing data


In [119]:
# train_labels['assessment_id'] = range(1,len(train_labels)+1)

# Merge train_labels info with event data info to look at accuracy groups with respect to
# different event info
train_order = pd.merge_ordered(left = train, right = train_labels, 
                 on = ['installation_id','game_session'], how='left')
train_order.sort_index()


### Fill in missing accuracy group values for all events and
### set index to variables that dataframe will be grouped by
# First set index and sort chronologically within an 
#   [installation_id, world] pair with timestamp in index
train_time = train_order.set_index(['installation_id','world','timestamp'])

# Delete the train_order object to conserve RAM 
del train_order

train_time = train_time.sort_index()

# Backfill NaN values of events around assessments, bounded by installation_id and world 
train_time = train_time.groupby(level=['installation_id','world']).bfill()

train_time.info()

# # 'Clip' type activties don't have a gametime, so we need to fill in a time.  Assume about 2min / clip.
train_time_clips = train_time[train_time['type'] == 'Clip'].replace(0,120000)['game_time']
train_time.loc[train_time['type'] == 'Clip','game_time'] = train_time_clips 

# Then fill the rest of the events with 0 for those events 
# not associated with installation_ids or game_sessions where an assessment took place.  
train_time = train_time.fillna(0)

# Add accuracy_group and type to index so you can do a multi-level sort
train_time = train_time.set_index(['game_session','accuracy_group','type'],append=True) # ,'assessment_id'

# Auto-dispatch the sum aggregation on multi-level group using the groupby method
# Essentially sum the amount of time for each type of gameplay for a particular installation_id ->
# world -> accuracy_group -> type of event to get an estimate of time spent in 
# each type of activity
train_time_agg = train_time.groupby( \
                 level=['installation_id','world','game_session']) \ # 'assessment_id',
                 .last()['game_time']

train_time_agg = train_time['game_time'].groupby(level=['installation_id','world',\
                                                        'type','accuracy_group'])\ # 'assessment_id',
                                                        .agg(game_time='sum')

SyntaxError: unexpected character after line continuation character (<ipython-input-119-a7c4e5fe773d>, line 42)

In [ ]:
# Now unstack the type column to get counts for each type of activity within each 
# [installation_id, world] pair
train_time = train_time_agg.unstack('type')

# Fill all those gameplay times that are empty with zeros so that the next step,
# the cumulative sum below, executes correctly. 
train_time = train_time.fillna(0)

# Calculate the cumulative sum of game time at assessment time in each type of gameplay for each 
# [installation_id world] pair
train_time = train_time['game_time'].groupby(level=['installation_id','world'])\
                                        .transform(pd.DataFrame.cumsum)

# Delete the train_time_agg object to conserve RAM on the Kaggle kernel
del train_time_agg

# Fill all [installation_id, world] rows that are missing an activity count with zeros
train_time = train_time.fillna(0)

# Now move the accuracy group out of the index to look at 
# correlation with type of activity counts
train_time.reset_index(level=3, inplace=True)

In [ ]:
# Now look at the time spent in each type of gameplay separated by 
# accuracy_group.  
sns.pairplot(train_time_scale,
             kind="scatter", 
             hue='accuracy_group')

The plot above shows a clear relationship between the cumulative amount of time spent on Activity and Clip type gameplays and the accuracy group of the child on the assessment.  Let's quantify this by performing a correlation.  

In [ ]:
train_corr = train_time_scale.corr()[['accuracy_group']]
display(train_corr)

As expected, both Activity and Clip cumulative gameplay time are correlated with the accuracy_group.  Let's now build a model using these features and see how we do.

In [ ]:
# Time to build models with the gameplay time features
# Create arrays for the features and the response variable
y = train_time['accuracy_group'].values
X = train_time.drop(['accuracy_group','Assessment'], axis=1).values

del train_time

In [ ]:
# Create a k-NN classifier with 4 neighbors, one for each accuracy_group
knn = KNeighborsClassifier(n_neighbors=4)

# Fit the classifier to the data
knn.fit(X, y)

In [ ]:
# Its important to use binary mode 
knnPickle = open('knn_pickle_file', 'wb') 

# source, destination 
pickle.dump(knn, knnPickle)                      

In [ ]:
# load the model from disk
knn = pickle.load(open('knn_pickle_file', 'rb'))

In [ ]:
# Print the accuracy
print(knn.score(X_test, y_test))

y_pred = knn.predict(X_test)

cv_scores = cross_val_score(knn, X, y, cv = 5)

# Print the 5-fold cross-validation scores
print(cv_scores)

print("Average 5-Fold CV Score: {}".format(np.mean(cv_scores)))
(len(y_test) - sum(y_pred != y_test))/ len(y_test)

Here we see that with just 3 features of cumulative gameplay time in each type of activity (excluding assessment time), we can get approximately 75-80% accuracy in predicting the child's performance on assessments.  

In [ ]:
# Now format and process the test set to make predictions

### Fill in missing accuracy group values for all events and
### set index to variables that dataframe will be grouped by
# First set index and sort chronologically within an 
#   [installation_id, world] pair with timestamp in index
test_time = test.set_index(['installation_id','world','timestamp'])

# Add a column, 'assessment_id', that keeps track of assessment 
# chronologically for each installation_id
test_time.loc[test_time['type']=='Assessment','assessment_id'] = \
                                               range(1,len(test_time[test_time['type']=='Assessment'])+1)

# del test

test_time = test_time.sort_index()

# Backfill NaN values of events that took place before assessments, these
# will be the events that impacted performance on those assessments.
test_time = test_time.groupby(level=['installation_id','world']).bfill()

# # 'Clip' type activities don't have a gametime, so we need to fill in a time.  Assume about 2min / clip.
test_time_clips = test_time[test_time['type'] == 'Clip'].replace(0,120000)['game_time']
test_time.loc[test_time['type'] == 'Clip','game_time'] = test_time_clips 

# Then fill the rest of the events with 0 for those events 
# not associated with installation_ids or game_sessions where an assessment took place.  
test_time = test_time.fillna(0)

# Add accuracy_group and type to index so you can do a multi-level sort
test_time = test_time.set_index(['game_session','assessment_id','type'],append=True)

# Auto-dispatch the sum aggregation on multi-level group using the groupby method
# Essentially sum the amount of time for each type of gameplay for a particular installation_id ->
# world -> accuracy_group -> type of event to get an estimate of time spent in 
# each type of activity
test_time_agg = test_time.groupby(level=['installation_id','world','assessment_id','game_session'])\
                .last()['game_time']


test_time_agg = test_time['game_time'].groupby(level=['installation_id','world','assessment_id','type'])\
                                      .agg(game_time='sum')


# Now unstack the type column to get counts for each type of activity with each 
# [installation_id, world] pair
test_time = test_time_agg.unstack('type')
del test_time_agg

# Fill all [installation_id, world] rows that are missing an activity count with zeros
test_time = test_time.fillna(0)

test_time = test_time['game_time'].groupby(level=['installation_id','world'])\
                                        .transform(pd.DataFrame.cumsum)


In [ ]:
# Create arrays for the features and the response variable
# X_final = test_time_scale.drop(['Assessment'], axis=1).values
X_final = test_time.drop(['Assessment'], axis=1).values

y_pred = knn.predict(X_final)

# Coerce the predictions to ints since this is required for a 
# correct submission format.
y_pred = y_pred.astype('int')

In [ ]:
# Put features and response variable into dataframe to be written to csv
sub = pd.DataFrame({'installation_id':test_time.index.get_level_values(0).values, 
                    'accuracy_group':y_pred})
                                         
# Pick the last assessment score for each installation_id    
sub = sub.groupby('installation_id').last()

# Move installation_id out of the index as required
# for the correct submission format.
sub.reset_index(inplace=True)

# Write to csv
sub.to_csv('submission.csv', index = False)